In [ ]:
from glob import glob
import rasterio
import xarray as xr
sza_files = sorted(glob('/home/imperatoren/work/VIIRS_S2_comparison/data/CMS_rejeu/*/*2024033*SatelliteZenithAngle.tif'))
for i,f in enumerate(sza_files):
    print(f)
    sza = xr.open_dataset(f).data_vars['band_data']
    # We consider that if there are no data we are outside the swath and if we are outside the swath it means that VZA are generally high and cannot be as low as 1
    if (sza==0).sum()>0 and (sza==1).sum() == 0:
        sza = sza.where(sza>0, 255)
    sza = sza.astype('u1')
    sza.to_netcdf(f.replace('SatelliteZenithAngle', 'SatelliteZenithAngleMod'), mode='w', encoding={'band_data': {"zlib": True}})

/home/imperatoren/work/VIIRS_S2_comparison/data/CMS_rejeu/202310/20231001_123430_npp_lzw_SatelliteZenithAngle.tif
<xarray.DataArray 'band_data' (band: 1, y: 3112, x: 4447)> Size: 14MB
array([[[48, 48, 48, ..., 88, 88, 88],
        [48, 48, 48, ..., 88, 88, 88],
        [48, 48, 48, ..., 88, 88, 88],
        ...,
        [93, 93, 93, ..., 67, 67, 67],
        [93, 93, 93, ..., 67, 67, 67],
        [93, 93, 93, ..., 67, 67, 67]]], dtype=uint8)
Coordinates:
  * band     (band) int64 8B 1
  * x        (x) float64 36kB -5.002 -4.998 -4.995 -4.992 ... 9.995 9.998 10.0
  * y        (y) float64 25kB 51.49 51.49 51.49 51.48 ... 41.01 41.01 41.0 41.0
Attributes:
    AREA_OR_POINT:  Area
    grid_mapping:   spatial_ref
/home/imperatoren/work/VIIRS_S2_comparison/data/CMS_rejeu/202310/20231002_121558_npp_lzw_SatelliteZenithAngle.tif
<xarray.DataArray 'band_data' (band: 1, y: 3112, x: 4447)> Size: 14MB
array([[[ 83,  83,  83, ...,  52,  52,  52],
        [ 83,  83,  83, ...,  52,  52,  52],
        

In [ ]:
import xarray as xr
from grids import georef_netcdf
import pyproj

def georef_netcdf(data_array: xr.DataArray | xr.Dataset, crs: pyproj.CRS) -> xr.Dataset | xr.Dataset:
    """
    The strict minimum to georeference in netCDF convention

    Enforce georeferencing for GDAL/QGIS
    https://github.com/pydata/xarray/issues/2288
    https://gis.stackexchange.com/questions/230093/set-projection-for-netcdf4-in-python
    """

    # dims = dim_name(crs=crs)
    data_array.coords["y"].attrs["axis"] = "Y"
    data_array.coords["x"].attrs["axis"] = "X"
    data_array.attrs["grid_mapping"] = "spatial_ref"

    georeferenced = data_array.assign_coords(coords={"spatial_ref": 0})
    georeferenced.coords["spatial_ref"].attrs["spatial_ref"] = crs.to_wkt()

    return georeferenced

fn = '/home/imperatoren/work/VIIRS_S2_comparison/data/CMS_samples/sample_composite_journalier/snowCoverComposite_noaa20_20250520_1200_00000_testlogr8220x.nc'
ds = xr.open_dataset(fn)
georef_netcdf(data_array=ds, crs=pyproj.CRS.from_epsg(4326)).to_netcdf(fn.replace('.nc', '_georef.nc'))

In [7]:
from view_angle_daily_composite import create_temporal_composite_meteofrance

compo = create_temporal_composite_meteofrance(daily_snow_cover_files=
                                      ['/home/imperatoren/work/VIIRS_S2_comparison/data/CMS_samples/sample_composite_journalier/frSnowCoverTif_noaa20_20250520_1137_38876_testlogr8220x.tif',
                                       '/home/imperatoren/work/VIIRS_S2_comparison/data/CMS_samples/sample_composite_journalier/frSnowCoverTif_noaa20_20250520_1305_38877_testlogr8220x.tif'
                                       ],
                                       daily_geometry_files=
                                       ['/home/imperatoren/work/VIIRS_S2_comparison/data/CMS_samples/sample_composite_journalier/szaSnowCoverTif_noaa20_20250520_1124_38876_testlogr8220x.tif',
                                        '/home/imperatoren/work/VIIRS_S2_comparison/data/CMS_samples/sample_composite_journalier/20231202_131154_npp_lzw_SatelliteZenithAngle_FAUX.tif'])

In [8]:
compo.to_netcdf('test_compo_20250520.nc')